#### Libraries

In [1]:
%%javascript
utils.load_extension("collapsible_headings/main")
utils.load_extension("hide_input/main")
utils.load_extension("autosavetime/main")
utils.load_extension("execute_time/ExecuteTime")
utils.load_extension("code_prettify/code_prettify")
utils.load_extension("scroll_down/main")
utils.load_extension("jupyter-js-widgets/extension")

<IPython.core.display.Javascript object>

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from category_encoders import TargetEncoder

plt.style.use("seaborn")
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, random_split
from sklearn.metrics import mean_squared_error

from nnet import ReadDataset, Net,ResNet
import time
from loss_functions import interval_score_loss

from pytorch_tabnet.tab_model import TabNetClassifier
from gauss_rank_scaler import GaussRankScaler
tic = time.time()
from models.lgbm import (compute_metrics, preprocess)

from tools.metrics import (
    get_avg_volumes,
)

In [3]:
def my_metric(pred, lower, upper):

    metric_pair = compute_metrics(
        preds=pred,
        lower=lower,
        upper=upper,
        y=val_y_raw,
        offset=val_offset,
        X=val_x_orig,
        avg_volumes=avg_volumes,
    )
    return metric_pair[0],metric_pair[1]

In [4]:
def postprocess_submission(submission_df, solve_submission_issues=True):

    join_on = ["country", "brand", "month_num"]
    keep = join_on + ["volume"]

    df_vol = pd.read_csv("../data/gx_volume.csv").loc[:, keep]

    both_ds = submission_df.merge(
        df_vol,
        on=join_on,
        how="left",
    )

    both_ds.loc[both_ds["volume"].notnull(), "prediction"] = both_ds[both_ds["volume"].notnull()]["volume"].values
    both_ds.loc[both_ds["volume"].notnull(), "pred_95_high"] = both_ds[both_ds["volume"].notnull()]["volume"].values + 0.01
    both_ds.loc[both_ds["volume"].notnull(), "pred_95_low"] = both_ds[both_ds["volume"].notnull()]["volume"].values - 0.01

    final_cols = join_on + ["pred_95_low", "prediction", "pred_95_high"]

    final_df =  both_ds.loc[:, final_cols]

    if solve_submission_issues:

        if (final_df.pred_95_low > final_df.pred_95_high).any():
            raise("Stop please, upper < lower")

        cond_lower_mean = final_df.pred_95_low > final_df.prediction
        if cond_lower_mean.any():
            print("Solving lower > mean")
            final_df.loc[cond_lower_mean, "prediction"] = \
                final_df.loc[cond_lower_mean, "pred_95_low"] + 0.01

        cond_upper_mean = final_df.prediction > final_df.pred_95_high
        if cond_upper_mean.any():
            print("Solving upper < mean")
            final_df.loc[cond_upper_mean, "prediction"] = \
                final_df.loc[cond_upper_mean, "pred_95_high"] - 0.01


    return final_df


In [5]:
def my_metric(pred, lower, upper):

    metric_pair = compute_metrics(
        preds=pred,
        lower=lower,
        upper=upper,
        y=val_y_raw,
        offset=val_offset,
        X=val_x_orig,
        avg_volumes=avg_volumes,
    )
    return metric_pair[0],metric_pair[1]

In [6]:
file_name = "linear_base"
offset_name = "last_before_3_after_0"
full_df = pd.read_csv("../data/gx_merged_lags_months.csv")
    # volume_features = pd.read_csv("data/volume_features.csv")
submission_df = pd.read_csv("../data/submission_template.csv")
train_tuples = pd.read_csv("../data/train_split.csv")
valid_tuples = pd.read_csv("../data/valid_split.csv")

 
# full_df = full_df.merge(volume_features, on=["country", "brand"])


full_df["volume_offset"] = (full_df["volume"] - full_df[offset_name]) / full_df[offset_name]
full_df = preprocess(full_df)

test_df = full_df[full_df.test == 1].copy().reset_index(drop=True)

real_full = full_df.copy()

full_df = full_df[full_df.test == 0]

train_df = full_df.merge(train_tuples, how="inner").reset_index(drop=True)
val_df = full_df.merge(valid_tuples, how="inner").reset_index(drop=True)

    # TODO: no need for calculation every time
avg_volumes = get_avg_volumes()

to_drop = ["volume", "volume_offset",'test','A','country_mean_before_24_after_0']
categorical_cols = [
        "country", "brand", "therapeutic_area", "presentation", "month_name",
        "month_country", "month_presentation", "month_area",
        "month_country_num", "month_presentation_num", "month_area_num",
        "month_month_num"
    ]

    # Prep data
train_x = train_df.drop(columns=to_drop)
train_y = train_df.volume_offset
train_offset = train_df[offset_name]



full_x = full_df.drop(columns=to_drop)
full_y = full_df.volume_offset
full_offset = full_df[offset_name]

real_full_x = real_full.drop(columns=to_drop)


val_x = val_df.drop(columns=to_drop)
val_x_orig = val_x
val_y = val_df.volume_offset
val_y_raw = val_df.volume
val_offset = val_df[offset_name]

test_x = test_df.drop(columns=to_drop)
test_offset = test_df[offset_name]

### Categorical

In [7]:
from category_encoders.one_hot import OneHotEncoder

In [8]:

categorical_cols = [
        "country", "brand", "therapeutic_area", "presentation", "month_name",
        "month_country", "month_presentation", "month_area",
        "month_country_num", "month_presentation_num", "month_area_num",
        "month_month_num"
    ]
te = OneHotEncoder(cols=categorical_cols)
te.fit(full_x,full_y)
#te.fit(train_x,train_y)

/opt/anaconda3/lib/python3.7/site-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


OneHotEncoder(cols=['country', 'brand', 'therapeutic_area', 'presentation',
                    'month_name', 'month_country', 'month_presentation',
                    'month_area', 'month_country_num', 'month_presentation_num',
                    'month_area_num', 'month_month_num'],
              drop_invariant=False, handle_missing='value',
              handle_unknown='value', return_df=True, use_cat_names=False,
              verbose=0)

In [9]:
full_x = te.transform(full_x)
train_x = te.transform(train_x)
val_x = te.transform(val_x)
test_x= te.transform(test_x)

real_full_x =te.transform(real_full_x)

### Scaling

In [10]:
scaler = GaussRankScaler()


In [11]:
#scaler.fit(train_x)
scaler.fit(real_full_x)

GaussRankScaler(copy=True, epsilon=0.0001, interp_copy=False,
                interp_kind='linear', n_jobs=None)

In [12]:
full_x = pd.DataFrame(scaler.transform(full_x),columns=full_x.columns)
train_x = pd.DataFrame(scaler.transform(train_x),columns=full_x.columns)
val_x = pd.DataFrame(scaler.transform(val_x),columns=full_x.columns)
test_x = pd.DataFrame(scaler.transform(test_x),columns=full_x.columns)


In [13]:
class ReadDataset(Dataset):
    """Read dataset."""

    def __init__(self, XX,yy):
        """
        Args:
            csv_file (str): Path to the csv file with the students data.

        """

        self.X = XX
        self.y = yy

    def __len__(self):
        return len(self.X)

    def __shape__(self):
        return self.X.shape[1]

    def __getitem__(self, idx):
        # Convert idx from tensor to list due to pandas bug (that arises when using pytorch's random_split)
        if isinstance(idx, torch.Tensor):
            idx = idx.tolist()

        self.X.iloc[idx].values
        self.y[idx]

        return [self.X.iloc[idx].values, self.y[idx]]

## NN

### Preprocess

In [14]:
trainset = ReadDataset(train_x,train_y)
testset = ReadDataset(val_x,val_y)


# Data loaders
trainloader = DataLoader(trainset, batch_size=100, shuffle=True)
# Test set

X_train = torch.tensor(trainset.X.values)
y_train = torch.tensor(trainset.y)

X_test = torch.tensor(testset.X.values)
y_test = torch.tensor(testset.y)

In [15]:
# Use gpu if available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


In [16]:
# Neural Network
nnet = Net(trainset.__shape__()).to(device)

In [17]:
# Optimizer
optimizer = optim.Adam(
    nnet.parameters(), lr=0.001, betas=(0.9, 0.999), eps=1e-08,  # weight_decay=0.0001
)

### Trainning

In [18]:
# Train the net
loss_per_iter = []
loss_per_batch = []


# Train the net
losses = []
auc_train = []
auc_test = []
metric_val = []
unc_val = []

# hyperparameteres
n_epochs = 50

for epoch in range(n_epochs):

    for i, (inputs, labels) in enumerate(trainloader):
        X = inputs.to(device)
        y = labels.to(device)


        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forwarde
        outputs = nnet(X.float())

        # Compute diff

        loss = interval_score_loss(outputs, y.float())

        # Compute gradient
        loss.backward()

        # update weights
        optimizer.step()

        # Save loss to plot

        losses.append(loss.item())

    if epoch % 5 == 0:
        print(epoch)
        auc_train.append(loss.cpu().detach().numpy())
        pred = nnet(X_test.float())
        auc_test.append(interval_score_loss(pred, y_test.float()))

        preds = torch.mean(nnet(X_test.float()), axis=1).cpu().detach().numpy()
        lower = nnet(X_test.float())[:, 0].cpu().detach().numpy()
        upper = nnet(X_test.float())[:, 1].cpu().detach().numpy()
        res = my_metric(preds, lower, upper)
        print(res)
        metric_val.append(res[0])
        unc_val.append(res[1])

        # Figure
        plt.figure()
        plt.plot(auc_train, label="train")
        plt.plot(auc_test, label="test")
        plt.plot(metric_val, label="Metric")
        plt.plot(unc_val, label="Uncertainty")
        plt.legend()
        plt.ylim([0, 100])
        plt.savefig("output/auc_NN.png")
        plt.savefig("output/auc_NN.svg", format="svg")
        plt.close()

        #
        path = "output/weights" + str(epoch) + ".pt"
        torch.save(nnet.state_dict(), path)

print("Elapsed time: ", np.abs(tic - time.time()))
print("done")

0
(14.756340860174568, 57.05058314736424)
5
(15.2303357067158, 57.10303926373913)
10
(16.38497677258529, 61.74076209829261)


KeyboardInterrupt: 

## Predictions Ensemble

In [ ]:
def curation_post(data):
    aux = data.copy()

    # Save arrays
    aux_low = aux[aux["pred_95_high"] < aux["pred_95_low"]].pred_95_low
    aux_high = aux[aux["pred_95_high"] < aux["pred_95_low"]].pred_95_high
    aux_index = aux[aux["pred_95_high"] < aux["pred_95_low"]].index

    # Modify
    aux.loc[aux_index, "pred_95_low"] = aux_high
    aux.loc[aux_index, "pred_95_high"] = aux_low
    
    
    if aux[aux["pred_95_high"] < aux["pred_95_low"]].shape[0]>0:
        print('If errors they should appear: ')
        print(aux[aux["pred_95_high"] < aux["pred_95_low"]].shape)
        print(aux[aux["pred_95_high"] < aux["pred_95_low"]].shape)

    preds_aux = np.mean([aux.pred_95_low, aux.pred_95_high], axis=0)
    
    aux['prediction'] = preds_aux
    return aux


In [ ]:
def predict_withNN(NN, data):
    # Make predictions
    preds = torch.mean(NN(X_test.float()), axis=1).detach().numpy()
    lower = NN(X_test.float())[:, 0].detach().numpy()
    upper = NN(X_test.float())[:, 1].detach().numpy()

    print(my_metric(preds, lower, upper))

    # Modify offset
    
    #preds = (preds + 1) * val_offset
    #lower = (lower + 1) * val_offset
    #upper = (upper + 1) * val_offset

    aux_data = data.copy()

    aux_data["prediction"] = preds
    aux_data["pred_95_low"] = lower
    aux_data["pred_95_high"] = upper

    aux_data = curation_post(aux_data)
    return aux_data

In [ ]:
nnet60 = Net(trainset.__shape__()).to(device)
nnet60.load_state_dict(torch.load("output/weights30.pt",map_location=torch.device('cpu')))

In [ ]:
nnet80 = Net(trainset.__shape__()).to(device)
nnet80.load_state_dict(torch.load("output/weights35.pt",map_location=torch.device('cpu')))

In [ ]:
nnet95 = Net(trainset.__shape__()).to(device)
nnet95.load_state_dict(torch.load("output/weights40.pt",map_location=torch.device('cpu')))

In [ ]:
nnet100 = Net(trainset.__shape__()).to(device)
nnet100.load_state_dict(torch.load("output/weights45.pt",map_location=torch.device('cpu')))

In [ ]:
n_60 = predict_withNN(nnet60,val_x_orig[['country','brand','month_num']])
n_80 = predict_withNN(nnet80,val_x_orig[['country','brand','month_num']])
n_95 = predict_withNN(nnet95,val_x_orig[['country','brand','month_num']])
n_100 = predict_withNN(nnet100,val_x_orig[['country','brand','month_num']])




In [ ]:
n_val_final = n_100.copy()

n_val_final.prediction = np.mean([n_60.prediction,
                                n_80.prediction,
                                n_95.prediction,
                                n_100.prediction],axis=0)

n_val_final.pred_95_low = np.mean([n_60.pred_95_low,
                                n_80.pred_95_low,
                                n_95.pred_95_low,
                                n_100.pred_95_low],axis=0)

n_val_final.pred_95_high = np.mean([n_60.pred_95_high,
                                n_80.pred_95_high,
                                n_95.pred_95_high,
                                n_100.pred_95_high],axis=0)

In [ ]:
my_metric(n_val_final.prediction,
         n_val_final.pred_95_low,
         n_val_final.pred_95_high)

In [ ]:
# One Hot
(11.703532084517162, 40.87996049931868)

In [ ]:
n_val_final.shape

In [ ]:
n_val_final.to_csv('output/valid_ensemble.csv',index=False)

## Predict Test

In [ ]:
def submission_predict(NN):

    submission_df = pd.read_csv("../data/submission_template.csv")

    preds = torch.mean(NN(torch.tensor(test_x.values).float()), axis=1).detach().numpy()
    lower = NN(torch.tensor(test_x.values).float())[:, 0].detach().numpy()
    upper = NN(torch.tensor(test_x.values).float())[:, 1].detach().numpy()

    preds = (preds + 1) * test_offset
    lower = (lower + 1) * test_offset
    upper = (upper + 1) * test_offset

    submission_df["pred_95_low"] = np.maximum(lower, 0)
    submission_df["pred_95_high"] = np.maximum(upper, 0)
    submission_df["prediction"] = np.maximum(preds, 0)
    submission_df = curation_post(submission_df)

    submission_df = postprocess_submission(submission_df)

    submission_df["pred_95_low"] = np.maximum(submission_df.pred_95_low, 0)
    submission_df["pred_95_high"] = np.maximum(submission_df.pred_95_high, 0)
    submission_df["prediction"] = np.maximum(submission_df.prediction, 0)

    e = submission_df[
        submission_df["pred_95_high"] < submission_df["pred_95_low"]
    ].shape[0]
    if e > 0:
        print("WARNING:ERORR, please debug")

    e = submission_df[
        submission_df["pred_95_low"] > submission_df["pred_95_high"]
    ].shape[0]
    if e > 0:
        print("WARNING:ERORR, please debug")

    e = submission_df[
        submission_df["prediction"] > submission_df["pred_95_high"]
    ].shape[0]
    if e > 0:
        print("WARNING:ERORR, please debug")

    e = submission_df[submission_df["prediction"] < submission_df["pred_95_low"]].shape[
        0
    ]
    if e > 0:
        print("WARNING:ERORR, please debug")
    return submission_df

In [ ]:
pred60 = submission_predict(nnet60)
pred80 = submission_predict(nnet80)
pred95 = submission_predict(nnet60)
pred100 = submission_predict(nnet80)

In [ ]:
pred_final = pred60.copy()

pred_final.prediction = np.mean(
    [pred60.prediction, pred80.prediction, pred95.prediction, pred100.prediction],
    axis=0,
)

pred_final.pred_95_low = np.mean(
    [pred60.pred_95_low, pred80.pred_95_low, pred95.pred_95_low, pred100.pred_95_low],
    axis=0,
)

pred_final.pred_95_high = np.mean(
    [
        pred60.pred_95_high,
        pred80.pred_95_high,
        pred95.pred_95_high,
        pred100.pred_95_high,
    ],
    axis=0,
)

In [ ]:
pred_final.describe()

In [ ]:
pred60.to_csv("../submissions/pred60_noPost.csv", index=False)

In [ ]:
pred_final.to_csv("../submissions/pred_final.csv", index=False)

# Pred final vanilla on val
(11.54, 40.81)

In [29]:
def postprocess_submission(submission_df, solve_submission_issues=True):

    join_on = ["country", "brand", "month_num"]
    keep = join_on + ["volume"]

    df_vol = pd.read_csv("../data/gx_volume.csv").loc[:, keep]

    both_ds = submission_df.merge(
        df_vol,
        on=join_on,
        how="left",
    )

    both_ds.loc[both_ds["volume"].notnull(), "prediction"] = both_ds[both_ds["volume"].notnull()]["volume"].values
    both_ds.loc[both_ds["volume"].notnull(), "pred_95_high"] = both_ds[both_ds["volume"].notnull()]["volume"].values + 0.01
    both_ds.loc[both_ds["volume"].notnull(), "pred_95_low"] = both_ds[both_ds["volume"].notnull()]["volume"].values - 0.01

    final_cols = join_on + ["pred_95_low", "prediction", "pred_95_high"]

    final_df =  both_ds.loc[:, final_cols]

    if solve_submission_issues:

        if (final_df.pred_95_low > final_df.pred_95_high).any():
            print(np.sum(final_df.pred_95_low > final_df.pred_95_high))
            raise("Stop please, upper < lower")

        cond_lower_mean = final_df.pred_95_low > final_df.prediction
        if cond_lower_mean.any():
            print("Solving lower > mean")
            print(np.sum(final_df.pred_95_low > final_df.prediction))
            final_df.loc[cond_lower_mean, "prediction"] = \
                final_df.loc[cond_lower_mean, "pred_95_low"] + 0.01

        cond_upper_mean = final_df.prediction > final_df.pred_95_high
        if cond_upper_mean.any():
            print("Solving upper < mean")
            final_df.loc[cond_upper_mean, "prediction"] = \
                final_df.loc[cond_upper_mean, "pred_95_high"] - 0.01


    return final_df


In [30]:
data = pd.read_csv('../data/SUBMISSION CORRECTED.csv')

In [31]:
postprocess_submission(data)

Solving lower > mean
140


,country,brand,month_num,pred_95_low,prediction,pred_95_high
0,country_1,brand_121,0,3.327128e+07,3.327128e+07,3.327128e+07
1,country_1,brand_121,1,1.276786e+07,1.276786e+07,1.276786e+07
2,country_1,brand_121,2,7.870697e+06,7.870697e+06,7.870697e+06
3,country_1,brand_121,3,6.978249e+06,6.978249e+06,6.978249e+06
4,country_1,brand_121,4,7.223489e+06,7.223489e+06,7.223489e+06
...,...,...,...,...,...,...
4579,country_9,brand_187,19,1.596853e+07,2.453214e+07,3.435900e+07
4580,country_9,brand_187,20,1.343837e+07,2.138781e+07,3.060052e+07
4581,country_9,brand_187,21,1.374823e+07,2.080449e+07,2.911921e+07
4582,country_9,brand_187,22,7.453335e+06,1.567176e+07,2.566832e+07
